In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo("KPZhX-9xWMA", width=800, height=600)

# Exploring NIH Funding

The NIH, as part of its open government initiative, provides information about extramural funding. [NIH Reporter](https://projectreporter.nih.gov/) is a web interface to the database containing information about all funded grants. The NIH also provides downloads of the grant information in either CSV or XML formats at [NIH ExPORTER](https://exporter.nih.gov/).

We will look at the 2015 funding to explore a little more about Pandas and to think about how we would design a smaller database from these data.

## File Formats

The data are stored in a CSV file that is contained within a [zip file](https://en.wikipedia.org/wiki/Zip_(file_format)). A zip file is a compressed archive of files. Since there are (potentially) multiple files within a zip file, we need a way of iterating over them. The ZipFile instance has a ``namelist`` method that provides a list of files in the archive. By prior exploration, I know that there is only one file in this archive.

* We will use Python's [urllib](https://docs.python.org/3/library/urllib.html) package to read the file over the web.
    * Remember ``urlopen`` creates a file-like object that reads data across the web the same as from a local file.
* We will use Python's [io.BytesIO](https://docs.python.org/3/library/io.html?highlight=io.bytesio#io.BytesIO) and the [zipfile](https://docs.python.org/3/library/zipfile.html) module to open the zip archive and extract individual files from it.
    * ``BytesIO`` takes a string (what we read with urlopen) and converts it into an object that looks like (behaves like) a file object.
* We will use [Pandas](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) to read the CSV file into a dataframe.



In [ ]:
from zipfile import ZipFile
import os
import pandas as pd
import matplotlib.pyplot as plt
from urllib.request import urlopen
from io import BytesIO

#### I had to use the 'latin-1' encoding to get the file to read properly

In [ ]:
url="https://exporter.nih.gov/CSVs/final/RePORTER_PRJ_C_FY2015.zip"

with urlopen(url) as f0:
    with ZipFile(BytesIO(f0.read())) as myzip:
        nih_data = pd.read_csv(myzip.open(myzip.namelist()[0]), 
                               encoding='latin-1')
nih_data.head()

#### What does a particular row of our data look like?

A couple of possible approaches. The first approach I thought of was with [``iloc``](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.iloc.html).

* iloc uses integer indexing to access data, so this will be most imilar to indexing a list or a numpy array. We can also use slicing

In [ ]:
nih_data.shape

In [ ]:
nih_data.iloc[0]

#### What are the column names?

In [ ]:
nih_data.columns

In [ ]:
nih_data["PI_NAMEs"].head(20)

## Database Design

### Design a relational database that would contain the following information

* PI name(s)
* Organization name
* Organization department (e.g. Radiology)
* Total cost
* Project title

What would the tables be? Data types?

#### We are going to use sqlite3 with an in memory database

#### First create a connection to the database and then a cursor

In [ ]:
import sqlite3 as sql
db= sql.connect(":memory:")
cur = db.cursor()

### Notes:

* ``Project_Title`` and ``Principle_Investigator`` are of type integer because they are going to be foreign keys to ``rowid`` which is an intger primary key for the PI and organiztion tables we will create
* We declare the foreign key relationship
    * Which variables are foreign keys and what table(column) do they refer to
* "CREATE TABLE IF NOT EXISTS" avoid throwing an exception when trying to create a table that already exists

In [ ]:
#Tables = Project; PI; Orgs
"""We are creating a set of tables; 
command for creating a sql table is create table if not exists"""
query = \
"""Create table if not exists project (Project_Title text, 
                                       Principle_Investigator int, 
                                       Organization int, 
                                       Department int,
                                       Total_Cost float,
                                       FOREIGN KEY (Principle_Investigator) REFERENCES PIs(rowid),
                                       FOREIGN KEY (Organization) REFERENCES organization(rowid),
                                       FOREIGN KEY (Department) REFERENCES department(rowid))"""
cur.execute(query)

#### Create the PI, Organization, and Department tables

* just have the PI table contain the name of the PIs. 
* just have the organization name
* just have the department name
* Other variables we might have wanted to create columns for were the State where the organization is located and the zip code, etc.

In [ ]:
query = """CREATE TABLE IF NOT EXISTS PI (name text)"""
cur.execute(query)

In [ ]:
query2 = """CREATE TABLE IF NOT EXISTS organization ( Organization_Name text)"""
cur.execute(query2)

In [ ]:
query3 = """CREATE TABLE IF NOT EXISTS department ( Department_Name text)"""
cur.execute(query3)

### Now we need to populate our tables

We start with our organization,  PI tables. What we want is a row for each unique organization (PI) in the DataFrame. We get these values with the `nih_data`'s ``unique`` method.

In [ ]:
orgs = nih_data['ORG_NAME'].dropna().unique()
PIs = nih_data['PI_NAMEs'].dropna().unique()
departments = nih_data['ORG_DEPT'].dropna().unique()
len(PIs), len(orgs), len(departments)


#### We used the ``executemany`` method to insert all the values with one call

* ``executemany`` expects a list of tuples. Each tuple represents the collection of values that are going to be inserted into a single row
* ``orgs`` and ``PIs`` are numy arrays which are list like, but each element is a single value not a tuple of a value
* We use list comprehension to put the values into a 1-tuple

In [ ]:
cur.executemany("""INSERT INTO organization values(?)""", 
                [(o,) for o in orgs])

#### Look at what we put into the table

In [ ]:
cur.execute("""SELECT rowid,  Organization_name FROM organization""")
cur.fetchmany(5)

#### Everything looks good from ``organization

#### Repeat the process to insert values into ``PIs``

In [ ]:
cur.executemany("""INSERT INTO PI values(?)""", [(p,) for p in PIs])

In [ ]:
cur.executemany("""INSERT INTO department values(?)""", [(d,) for d in departments])

In [ ]:
cur.execute("""SELECT rowid,name FROM PI""")
cur.fetchmany(10)

## Populating the ``project`` database

* Populating the ``project`` database is more complicated beause what we need to insert into the row is not the names of the PI or the organizations but the **foreign keys** (the rowid for those names in their respective tables.

* For any given name (PI or organization) we need to query the appropriate table and get the rowid for that name.
     * This will be done with a ``SELECT`` statement with a ``WHERE`` clause
     * For example, ``SELECT rowid FROM PIs WHERE NAME='PARKER, DENNIS L.;'``

#### Best approach is to let sqlite3 do the interpolation/substitution for us

In [ ]:
for pi in PIs[:10]:
    cur.execute("""SELECT rowid FROM PI WHERE NAME=?""",(pi,))
    rslt = cur.fetchone()[0]
    print(pi, rslt)

### Putting it all together

In [ ]:
for _, row in nih_data.fillna("").iterrows():
    project_title = row['PROJECT_TITLE']
    total_cost = row['TOTAL_COST']
    pi_names = row['PI_NAMEs']
    org_name = row['ORG_NAME']
    dept_name = row['ORG_DEPT']
    
    cur.execute("""SELECT rowid FROM PI WHERE NAME=?""", (pi_names,))
    try:
        pi = pi_rowid = cur.fetchone()[0]
    except TypeError:
        pi = None
    try:
        cur.execute("""SELECT rowid FROM organization WHERE Organization_Name=?""", (org_name,))
        org = cur.fetchone()[0]
    except TypeError:
        org = None
    try:
        cur.execute("""SELECT rowid FROM department WHERE Department_Name=?""", (dept_name,))
        dept = cur.fetchone()[0]
    except TypeError:
        dept = None
    if pi and org and dept:
        cur.execute("""INSERT INTO project VALUES(?,?,?,?,? )""",(project_title, pi, org, dept, total_cost))

In [ ]:
row

In [ ]:
cur.execute("""SELECT * FROM project""")
len(cur.fetchall())

In [ ]:
cur.execute("""SELECT * FROM project""")

In [ ]:
for r in cur.fetchmany(5):
    print(r)

#### We need to do joins to get the names corresponding to our foreign key values

In [ ]:
query=\
"""SELECT project.Project_Title as Project_Title, 
          project.Total_Cost as Total_Cost,
          PI.Name as PI_Name,
          organization.Organization_Name as Organization_Name,
          department.Department_Name as Department_Name
          
   FROM project 
        INNER JOIN PI 
            ON 
                project.Principle_Investigator = PI.rowid
        INNER JOIN organization 
            ON
                project.Organization = organization.rowid
        INNER JOIN department
            ON
               project.Department = department.rowid"""

pd.read_sql(query, db)

In [ ]:
YouTubeVideo("qYNEQC01A88", width=800, height=600)

## Is Our Solution First Normal Form?

### No because we have multiple names in the PI field

### Here is one solution for creating a first normal form of our data

### Get Individual Names

With multiple principal investigators, names are separated by semi-colons with one person specified as the contact. We can do some text processing to get a unique set of names.

In [ ]:
names = [n.strip() for n in " ".join(PIs).replace(" (contact)","").split(";") if n.strip()]
print(len(names))
names = set(names)
print(len(names))
print(list(names)[:10])

### We need to drop our existing `project` table since we are redefining it

In [ ]:
db.execute("DROP TABLE project;")

### Define a new `project` table

In [ ]:
query = \
"""Create table if not exists project (Project_Title text, 
                                       Organization int, 
                                       Department int,
                                       Total_Cost float,
                                       FOREIGN KEY (Organization) REFERENCES organization(rowid),
                                       FOREIGN KEY (Department) REFERENCES department(rowid))"""
cur.execute(query)

### Delete all the rows from `PI` since we will be repopulating it with our individual names

In [ ]:
cur.execute("""DELETE FROM PI""")

### Insert our new names into `PI`

In [ ]:
cur.executemany("""INSERT INTO PI values(?)""", [(p,) for p in names])

### Create a new table `investigators` which connects PIs with particular projects

In [ ]:
query = \
"""Create table if not exists investigators (Project int, 
                                       Investigator int, 
                                       FOREIGN KEY (Project) REFERENCES project(rowid),
                                       FOREIGN KEY (Investigator) REFERENCES PI(rowid))"""
cur.execute(query)

In [ ]:
cur.execute("""DELETE FROM investigators""")

### Populate Tables
#### Iterate through DataFrame and populate tables
* For each row we need to iterate through each PI

In [ ]:
for _, row in nih_data[:100].fillna("").iterrows():
    project_title = row['PROJECT_TITLE']
    total_cost = row['TOTAL_COST']
    pi_names = row['PI_NAMEs']
    org_name = row['ORG_NAME']
    dept_name = row['ORG_DEPT']
    

    try:
        cur.execute("""SELECT rowid FROM organization WHERE Organization_Name=?""", (org_name,))
        org = cur.fetchone()[0]
    except TypeError:
        org = None
    try:
        cur.execute("""SELECT rowid FROM department WHERE Department_Name=?""", (dept_name,))
        dept = cur.fetchone()[0]
    except TypeError:
        dept = None
    if org and dept:
        cur.execute("""INSERT INTO project VALUES(?,?,?,? )""",(project_title, org, dept, total_cost))
    
    cur.execute("""SELECT last_insert_rowid()""")
    pid = cur.fetchone()[0]

    pis = [n.strip() for n in pi_names.replace(" (contact)","").split(";") if n.strip() and n != '"']

    for p in pis:
        cur.execute("""SELECT rowid FROM PI WHERE NAME=?""", (p,))
        pi = cur.fetchone()[0]
        cur.execute("""INSERT INTO investigators VALUES (?, ?)""", (pid, pi))


### Do a query to put our data back together

In [ ]:
query=\
"""SELECT 
          project.Project_Title as Project_Title, 
          project.Total_Cost as Total_Cost,
          organization.Organization_Name as Organization_Name,
          department.Department_Name as Department,
          PI.name as investigator
          
   FROM investigators 

        INNER JOIN project
            ON
                investigators.Project = project.rowid
        INNER JOIN PI
            ON
                investigators.Investigator = PI.rowid
        INNER JOIN organization 
            ON
                project.Organization = organization.rowid
        INNER JOIN department
            ON
                project.Department = department.rowid
"""

pd.read_sql(query, db)#.sort_values(by="Total_Cost", ascending=False)